In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import seaborn as sns
import datetime as dt
import os
import re
import os, sys, importlib, math, string
import connector as hk

In [2]:
# read in csv of portfolio analysis
table = pd.read_csv('clickshare_stage_one_funnel.csv')

In [3]:
table.head(20)

,Unnamed: 0,%spendhighimprshare,KS,cost,cost_on_high_imprshare,portfolio_id,ports
0,3,1.000000,KS5103,586548.959778,586548.959778,21.0,21.0
1,0,0.976331,KS3316,565199.531829,551821.620163,3.0,3.0
2,0,1.000000,KS7156,257129.555176,257129.555176,54.0,54.0
3,0,1.000000,KS3651,212660.249023,212660.249023,1.0,1.0
4,4,1.000000,KS3736,84847.439682,84847.439682,7.0,7.0
5,12,1.000000,KS5233,78594.990540,78594.990540,62.0,62.0
6,8,1.000000,KS3758,76014.430351,76014.430351,12.0,12.0
7,23,0.995731,KS5015,61925.630018,61661.290018,339.0,339.0
8,11,1.000000,KS3331,60990.399916,60990.399916,787.0,787.0
9,5,0.966403,KS7112,62621.499928,60517.579928,35.0,35.0


## Find cost at 90, 95, and 98 percent clickshare

In [4]:
def port_analysis(table, port_cost_threshold, start_date, end_date):
    #KS = string, portfolio _id is float one decimal place(e.g. 47.0), start_date and end_date are strings(YYYY-MM-DD)
    
    """for portfolios with a high amount of spend on high impressionshare campaigns and a significant amount of cost
       - generate implied cpc's for each affcode
       - calculate clickshare for max cpc's within a range of the min cpc affcode to the max cpc affcode
       - calculate costs for max cpc's within that range
       - generate a linear approximation of cost at key clickshare levels

       
       """
    
    table = table[table['cost'] > port_cost_threshold]
    #create list of KS's and portfolio id's
    lst = zip(table['KS'],table['portfolio_id'])
    
    #key clickshare levels
    Cts = [.90,.95,.98]
    
    #empty lists to contain cost approximations for each clickshare level
    Yts = [[],[],[]]
    
    #for each KS portfolio pair:
    for port in lst:
        KS = port[0][2:]
        portfolio_id = port[1]
    
        #connection to appropriate KS
        host = 'eclidb'+KS+'.kenshooprd.local'
        _mysql_connector = __import__('mysql.connector').connector if sys.version_info.major == 2 else __import__('pymysql')
        cnx = _mysql_connector.connect(user='query',password='query', database='kazaam',host=host)
    
        #pull campaign ids for campaigns that are members of the portfolio during time interval
        querycampaigns = f'''
        select campaign_id
        from portfolio_members 
        where portfolio_id = '{portfolio_id}'
        and start_date < '{start_date}'
        and end_date is NULL
        '''
        CampaignList = pd.read_sql(querycampaigns, con=cnx).campaign_id
        CampaignList_SQL = ','.join(map(str, CampaignList))
    
    
    
        #pull affcode level data
        queryPerfAffc =f''' 
        select *
               from performance_by_affcode
               where campaign_id in ({CampaignList_SQL})
               and date BETWEEN '{start_date}' and '{end_date}'
               and cost > 0
               
        '''
        PerfAffc = pd.read_sql(queryPerfAffc, con=cnx)
        dff = pd.DataFrame(PerfAffc)
    
        dft = dff.groupby('affcode').agg({'clicks': 'sum', 'cost': 'sum'})
        #Calculate implied cpc for each affcode
        dft['cpc'] = dft['cost']/dft['clicks']
        #Create an array of evenly spaced numbers ranging from the minimum implied cpc to the maximum implied cpc
        space = np.linspace(dft['cpc'].min(), dft['cpc'].max(), num = 100)
        clickshare = []
        cost = []
        avg_cpc = []
        savings = []
        for x in space:
            #Definition of cumulative distribution function
            clickshare.append(dft[dft['cpc'] <= x]['clicks'].sum()/dft['clicks'].sum())
            #cumulative cost at each max cpc
            cost.append(dft[dft['cpc'] <= x]['cost'].sum())
            #average cpc at each max cpc
            avg_cpc.append(dft[dft['cpc'] <= x]['cost'].sum()/ dft[dft['cpc'] <= x]['clicks'].sum())
            #savings at each max cpc
            savings.append(dft['cost'].sum() - dft[dft['cpc'] <= x]['cost'].sum())
        
        #linear approximation of cost for key impressionshare thresholds: 90%, 95%, 98%
        for j in range(len(Cts)):
            #in case of null
            null_indic = 1
            
            '''for each key clickshare threshold, find clickshare directly above and below threshold, costs
            associated with those clickshares, and then linearly approximate the cost value at the threshold
            '''
            
            for i in range(len(clickshare)):
                if clickshare[i] == Cts[j]:
                    Yt = cost[i]
                    Yts[j].append(Yt)
                    null_indic = 0
                    break
                else:
                    if clickshare[i] > Cts[j]:
                        '''if threshold exists between clickshare[i-1] and clickshare[i], find *a* such that
                        (a * clickshare[i-1] + (1- a) * clickshare[i]) = threshold'''
                        lower_bound_i = i - 1
                        lower_bound_clickshare = clickshare[i-1]
                        upper_bound_i = i
                        upper_bound_clickshare = clickshare[i]
                        
                        a = (Cts[j] - clickshare[i])/ (clickshare[i-1] - clickshare[i])
                        # use *a* to estimate the cost at threshold
                        
                        Yt = a * cost[i-1] + (1-a) * cost[i]
                        Yts[j].append(Yt)
                        null_indic = 0
                        break
            if null_indic == 1:
                    Yts[j].append(0)
                
         
    table['cost_of_90%_cs'] = Yts[0]
    table['cost_of_95%_cs'] = Yts[1]        
    table['cost_of_98%_cs'] = Yts[2]          
    
    table['%cost_of_90%_cs'] = table['cost_of_90%_cs']/table['cost_on_high_imprshare']
    table['%cost_of_95%_cs'] = table['cost_of_95%_cs']/table['cost_on_high_imprshare']      
    table['%cost_of_98%_cs'] = table['cost_of_98%_cs']/table['cost_on_high_imprshare']
    table = table.drop('ports', axis = 1)
    cols = list(df.columns.values)
    table = table[['%spendhighimprshare', 'KS', 'portfolio_id', 'cost', 'cost_on_high_imprshare',
        'cost_of_90%_cs', '%cost_of_90%_cs','cost_of_95%_cs', '%cost_of_95%_cs','cost_of_98%_cs',
         '%cost_of_98%_cs']]
    return table
        

In [5]:
table = port_analysis(table, 20000, '2019-10-01', '2019-10-31' )
table

/Users/michaelpo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in double_scalars
/Users/michaelpo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in double_scalars
/Users/michaelpo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/michaelpo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

,%spendhighimprshare,KS,portfolio_id,cost,cost_on_high_imprshare,cost_of_90%_cs,%cost_of_90%_cs,cost_of_95%_cs,%cost_of_95%_cs,cost_of_98%_cs,%cost_of_98%_cs
0,1.000000,KS5103,21.0,586548.959778,586548.959778,340095.286343,0.579824,424306.531919,0.723395,521651.988944,0.889358
1,0.976331,KS3316,3.0,565199.531829,551821.620163,442890.830125,0.802598,494460.510137,0.896051,525790.909444,0.952828
2,1.000000,KS7156,54.0,257129.555176,257129.555176,208425.481425,0.810585,227782.761769,0.885868,241825.218164,0.940480
3,1.000000,KS3651,1.0,212660.249023,212660.249023,90334.328174,0.424782,137271.641603,0.645497,167791.553306,0.789012
4,1.000000,KS3736,7.0,84847.439682,84847.439682,68524.621037,0.807622,74969.166292,0.883576,80855.670817,0.952954
5,1.000000,KS5233,62.0,78594.990540,78594.990540,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,1.000000,KS3758,12.0,76014.430351,76014.430351,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.995731,KS5015,339.0,61925.630018,61661.290018,36428.851116,0.590790,47655.382946,0.772857,54391.302044,0.882098
8,1.000000,KS3331,787.0,60990.399916,60990.399916,46329.889075,0.759626,52083.522936,0.853963,56724.855782,0.930062
9,0.966403,KS7112,35.0,62621.499928,60517.579928,42829.810771,0.707725,48006.736566,0.793269,53216.942960,0.879363
